In [1]:
import time
import os
from Research.backtest.bt import BTDaily
from Research.feature.ft import FeatureAnalysis
import Research.utils.namespace as namespace
import Research.utils.normalization as norm
from Platform.database.mysql import MysqlAPI
from Platform.utils.persistence import convert_to_standard_daily_feature_csv, convert_to_standard_daily_feature_par
from Platform.config.mysql_info import FACTOR_LIB_MYSQL_TIO
import DataAPI
from os import walk
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
from progressbar import ProgressBar
from tqdm import tqdm
import multiprocessing as mp
import sys
sys.path.append("/home/wuwenjun/jupyter_code/Shannon/AlphaNet/packages/")

In [7]:
start_date = "2015-01-01"
end_date = "2021-06-01"
target = "10d_ret"

In [8]:
time_list_old = DataAPI.get_trading_days(start_date, end_date)
time_list = []
for date in time_list_old:
    time_list.append(str(date.year) + "-"+ str(date.month).zfill(2) + "-"+str(date.day).zfill(2))
time_list

['2015-01-05',
 '2015-01-06',
 '2015-01-07',
 '2015-01-08',
 '2015-01-09',
 '2015-01-12',
 '2015-01-13',
 '2015-01-14',
 '2015-01-15',
 '2015-01-16',
 '2015-01-19',
 '2015-01-20',
 '2015-01-21',
 '2015-01-22',
 '2015-01-23',
 '2015-01-26',
 '2015-01-27',
 '2015-01-28',
 '2015-01-29',
 '2015-01-30',
 '2015-02-02',
 '2015-02-03',
 '2015-02-04',
 '2015-02-05',
 '2015-02-06',
 '2015-02-09',
 '2015-02-10',
 '2015-02-11',
 '2015-02-12',
 '2015-02-13',
 '2015-02-16',
 '2015-02-17',
 '2015-02-25',
 '2015-02-26',
 '2015-02-27',
 '2015-03-02',
 '2015-03-03',
 '2015-03-04',
 '2015-03-05',
 '2015-03-06',
 '2015-03-09',
 '2015-03-10',
 '2015-03-11',
 '2015-03-12',
 '2015-03-13',
 '2015-03-16',
 '2015-03-17',
 '2015-03-18',
 '2015-03-19',
 '2015-03-20',
 '2015-03-23',
 '2015-03-24',
 '2015-03-25',
 '2015-03-26',
 '2015-03-27',
 '2015-03-30',
 '2015-03-31',
 '2015-04-01',
 '2015-04-02',
 '2015-04-03',
 '2015-04-07',
 '2015-04-08',
 '2015-04-09',
 '2015-04-10',
 '2015-04-13',
 '2015-04-14',
 '2015-04-

In [12]:
import multiprocessing as mp
import time
from tqdm import tqdm
pool = mp.Pool(10)
t1 = time.time()
result = {}
def test(i):
    data = DataAPI.get_stock_daily_return(i, ti=0, tp=31, price='vwap', period=["10d"], freq='1min').reset_index()
    data['timestamp'] += pd.to_timedelta(5,"m")
    return data.set_index(["timestamp","ticker"])
for i in tqdm(time_list):
    result[i]  = (pool.apply_async(test, args=(i,)))   #维持执行的进程总数为10，当一个进程执行完后启动一个新进程.
pool.close()
pool.join()
t2 = time.time()
print("并行执行时间：", int(t2 - t1))

100%|██████████| 1560/1560 [00:00<00:00, 5834.22it/s]


并行执行时间： 3


In [13]:
target_df = []
for i in tqdm(result):
    target_df.append(result[i].get())
target_df = pd.concat(target_df,axis=0)
target_df.drop("trading_status",inplace=True,axis=1)
target_df

100%|██████████| 1560/1560 [00:00<00:00, 309718.56it/s]


10d_ret
timestamp           ticker          
2015-01-05 09:30:00 000001 -0.104853
                    000002 -0.136266
                    000004  0.018452
                    000005  0.000000
                    000006 -0.131916
...                              ...
2021-06-01 09:30:00 688699 -0.041999
                    688777 -0.060290
                    688788 -0.065377
                    688819 -0.031494
                    688981 -0.028147

[5281482 rows x 1 columns]

In [14]:
df = target_df.unstack()
df = (df.T - df.mean(axis=1)) / df.std(axis=1)
df = df.T.stack()
df = pd.DataFrame(df,columns=[target])
# to date time
df.reset_index(inplace=True)
df['timestamp'] = pd.to_datetime(df['timestamp'])
df.rename(columns = {"ticker":"symbol",target : target + "_standscalar"},inplace=True)
df

,timestamp,symbol,10d_ret_standscalar
0,2015-01-05 09:30:00,000001,-1.232455
1,2015-01-05 09:30:00,000002,-1.538025
2,2015-01-05 09:30:00,000004,-0.032981
3,2015-01-05 09:30:00,000005,-0.212472
4,2015-01-05 09:30:00,000006,-1.495711
...,...,...,...
5281477,2021-06-01 09:30:00,688699,-0.420129
5281478,2021-06-01 09:30:00,688777,-0.602658
5281479,2021-06-01 09:30:00,688788,-0.653424
5281480,2021-06-01 09:30:00,688819,-0.315297


In [15]:
convert_to_standard_daily_feature_csv("10d_ret_standscalar", df, "/home/ShareFolder/feature_platform/ti0/wuwenjun")

In [ ]:
from Research.feature.ft import FeatureAnalysis
import Research.utils.namespace as namespace
configs = namespace.load_namespace(r'/home/ShareFolder/lgc/Modules/Research/config/feature_bt_template')
FT = FeatureAnalysis(configs, feature_path=r"/home/ShareFolder/feature_platform")
FT.load_feature_from_file(["10d_ret_standscalar"], "2015-01-01", "2022-01-01", timedelta=None)
a = FT.feature_data
a

Loading Feature...:  66%|██████████████████████████████████████████████████████████▏                             | 1127/1705 [00:03<00:01, 421.31it/s]

In [21]:
a[a["ticker"] == "000001"]

,timestamp,ticker,10d_ret_standscalar
874,2015-01-28 09:30:00,000001,0.097933
4815,2015-01-07 09:30:00,000001,-1.211547
7221,2015-03-16 09:30:00,000001,-1.065587
8901,2015-01-27 09:30:00,000001,-0.192831
12036,2015-03-18 09:30:00,000001,-0.704107
...,...,...,...
5019386,2021-05-19 09:30:00,000001,-0.603337
5021125,2021-05-14 09:30:00,000001,0.278034
5027630,2021-05-13 09:30:00,000001,0.343195
5037923,2021-05-27 09:30:00,000001,-0.210223


In [20]:
from sklearn.preprocessing import StandardScaler, MinMaxScaler
ssr = StandardScaler()
mmr = MinMaxScaler(feature_range=(-10,10))
columns = target_df.columns
final_list = []
for name,group in tqdm(target_df.groupby("timestamp")):
    fina_df = group.copy()
    fina_df[[str(i) +"_norm_standard_scale" for i in group.columns]] = norm.standard_scale(group)
#     fina_df[[str(i) +"_StandardScaler" for i in group.columns]] = ssr.fit_transform(group)
#     fina_df[[str(i) +"_MinMaxScaler_10" for i in group.columns]] = mmr.fit_transform(group)
#     fina_df[[str(i) +"_StandardScale_MinMax_10" for i in group.columns]] = mmr.fit_transform(ssr.fit_transform(group))
    final_list.append(fina_df)
target_df = pd.concat(final_list,axis=0)
target_df

100%|██████████| 1560/1560 [00:05<00:00, 283.62it/s]


In [64]:
final_list = []
for column in tqdm(target_df.columns):
    a = pd.DataFrame(target_df[column])
    return_list = []
    for name,group in a.groupby("timestamp"):
        group[str(column) +"norm_standard_scale"] = norm.standard_scale(group)
        return_list.append(group)
    return_list = pd.concat(return_list,axis=0)
    final_list.append(return_list)
final_list = pd.concat(final_list,axis=1)
final_list

100%|██████████| 6/6 [00:51<00:00,  8.63s/it]


1d_ret  1d_retnorm_standard_scale    2d_ret  \
timestamp           ticker                                                  
2015-01-05 09:30:00 000001 -0.003120                  -0.541929 -0.031001   
                    000002 -0.001994                  -0.503045 -0.019423   
                    000004  0.014671                   0.072339  0.048657   
                    000005  0.000000                  -0.434206  0.000000   
                    000006 -0.040209                  -1.822556 -0.039435   
...                              ...                        ...       ...   
2021-06-01 09:30:00 688699 -0.057854                  -2.225986 -0.088197   
                    688777 -0.035939                  -1.456454 -0.049675   
                    688788 -0.031378                  -1.296282 -0.048310   
                    688819  0.025589                   0.704046  0.011450   
                    688981 -0.011507                  -0.598535 -0.023025   

                            2d_retnorm_standard_scale    3d_ret  \
timestamp           ticker                                        
2015-01-05 09:30:00 000001                  -1.584587 -0.036996   
                    000002                  -1.277469 -0.041521   
                    000004                   0.528394  0.065601   
                    000005                  -0.762266  0.000000   
                    000006                  -1.808308 -0.042880   
...                                               ...       ...   
2021-06-01 09:30:00 688699                  -2.134933 -0.089355   
                    688777                  -1.227989 -0.066356   
                    688788                  -1.195837 -0.066576   
                    688819                   0.211101  0.004278   
                    688981                  -0.600554 -0.038161   

                            3d_retnorm_standard_scale    4d_ret  \
timestamp           ticker                                        
2015-01-05 09:30:00 000001                  -1.475431 -0.067138   
                    000002                  -1.573327 -0.070923   
                    000004                   0.743830  0.071498   
                    000005                  -0.675178  0.000000   
                    000006                  -1.602709 -0.053565   
...                                               ...       ...   
2021-06-01 09:30:00 688699                  -1.764211 -0.008459   
                    688777                  -1.307274 -0.011289   
                    688788                  -1.311633 -0.043054   
                    688819                   0.096075  0.060774   
                    688981                  -0.747091 -0.011591   

                            4d_retnorm_standard_scale    5d_ret  \
timestamp           ticker                                        
2015-01-05 09:30:00 000001                  -1.691426 -0.070556   
                    000002                  -1.758539 -0.097998   
                    000004                   0.766915  0.038417   
                    000005                  -0.500910  0.000000   
                    000006                  -1.450746 -0.085893   
...                                               ...       ...   
2021-06-01 09:30:00 688699                  -0.221912 -0.079010   
                    688777                  -0.269707 -0.028392   
                    688788                  -0.806231 -0.060163   
                    688819                   0.947432  0.056411   
                    688981                  -0.274807 -0.013660   

                            5d_retnorm_standard_scale   10d_ret  \
timestamp           ticker                                        
2015-01-05 09:30:00 000001                  -1.317890 -0.104853   
                    000002                  -1.761349 -0.136266   
                    000004                   0.443097  0.018452   
                    000005                  -0.177714  0.000000   
          

In [78]:
import sys
sys.path.append("/home/wuwenjun/jupyter_code/Shannon/AlphaNet/packages/")
from AlphaNet.Data import DataLoader
trainloader = DataLoader()
trainloader.load_data_from_file(alpha_name = "10d_ret",end_date = "2022-01-01",data_path="/home/wuwenjun/feature_platform/ti0/wuwenjun/")

100%|██████████| 1705/1705 [00:12<00:00, 139.32it/s]


10d_ret
timestamp           ticker          
2015-01-05 09:30:00 000001 -0.104853
                    000002 -0.136266
                    000004  0.018452
                    000005  0.000000
                    000006 -0.131916
...                              ...
2021-06-01 09:30:00 688699 -0.041999
                    688777 -0.060290
                    688788 -0.065377
                    688819 -0.031494
                    688981 -0.028147

[5281482 rows x 1 columns]

In [88]:
trainloader.feature_data

,timestamp,symbol,10d_ret_test
0,2015-01-05 09:30:00,000001,-0.104853
1,2015-01-05 09:30:00,000002,-0.136266
2,2015-01-05 09:30:00,000004,0.018452
3,2015-01-05 09:30:00,000005,0.000000
4,2015-01-05 09:30:00,000006,-0.131916
...,...,...,...
5281477,2021-06-01 09:30:00,688699,-0.041999
5281478,2021-06-01 09:30:00,688777,-0.060290
5281479,2021-06-01 09:30:00,688788,-0.065377
5281480,2021-06-01 09:30:00,688819,-0.031494


In [89]:
convert_to_standard_daily_feature_csv("10d_ret_test", trainloader.feature_data, r'/home/wuwenjun/factor_lib/ti0/wuwenjun')

Process ForkPoolWorker-787:
Process ForkPoolWorker-778:
Process ForkPoolWorker-777:
Process ForkPoolWorker-794:
Process ForkPoolWorker-793:
Process ForkPoolWorker-792:
Process ForkPoolWorker-796:
Process ForkPoolWorker-785:
Process ForkPoolWorker-786:
Process ForkPoolWorker-789:
Process ForkPoolWorker-791:
Process ForkPoolWorker-779:
Process ForkPoolWorker-795:
Process ForkPoolWorker-802:
Process ForkPoolWorker-800:
Process ForkPoolWorker-797:
Process ForkPoolWorker-781:
Process ForkPoolWorker-798:
Process ForkPoolWorker-780:
Process ForkPoolWorker-784:
Process ForkPoolWorker-782:
Process ForkPoolWorker-790:
Process ForkPoolWorker-783:
Process ForkPoolWorker-788:
Process ForkPoolWorker-801:
Process ForkPoolWorker-799:
Process ForkPoolWorker-803:
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
  File "/usr/local/miniconda/lib/python3.7/multiprocessing/process.py", line 297, in _bootstrap
    self.run()
Traceback (most recent call 

  File "/usr/local/miniconda/lib/python3.7/multiprocessing/process.py", line 99, in run
    self._target(*self._args, **self._kwargs)
  File "/usr/local/miniconda/lib/python3.7/multiprocessing/process.py", line 99, in run
    self._target(*self._args, **self._kwargs)
  File "/usr/local/miniconda/lib/python3.7/multiprocessing/queues.py", line 351, in get
    with self._rlock:
  File "/usr/local/miniconda/lib/python3.7/multiprocessing/process.py", line 99, in run
    self._target(*self._args, **self._kwargs)
  File "/usr/local/miniconda/lib/python3.7/multiprocessing/pool.py", line 110, in worker
    task = get()
  File "/usr/local/miniconda/lib/python3.7/multiprocessing/pool.py", line 110, in worker
    task = get()
  File "/usr/local/miniconda/lib/python3.7/multiprocessing/pool.py", line 110, in worker
    task = get()
  File "/usr/local/miniconda/lib/python3.7/multiprocessing/queues.py", line 351, in get
    with self._rlock:
  File "/usr/local/miniconda/lib/python3.7/multiprocessing/p

  File "/usr/local/miniconda/lib/python3.7/multiprocessing/synchronize.py", line 95, in __enter__
    return self._semlock.__enter__()
  File "/usr/local/miniconda/lib/python3.7/multiprocessing/synchronize.py", line 95, in __enter__
    return self._semlock.__enter__()
  File "/usr/local/miniconda/lib/python3.7/multiprocessing/synchronize.py", line 95, in __enter__
    return self._semlock.__enter__()
  File "/usr/local/miniconda/lib/python3.7/multiprocessing/synchronize.py", line 95, in __enter__
    return self._semlock.__enter__()
  File "/usr/local/miniconda/lib/python3.7/multiprocessing/synchronize.py", line 95, in __enter__
    return self._semlock.__enter__()
  File "/usr/local/miniconda/lib/python3.7/multiprocessing/synchronize.py", line 95, in __enter__
    return self._semlock.__enter__()
  File "/usr/local/miniconda/lib/python3.7/multiprocessing/synchronize.py", line 95, in __enter__
    return self._semlock.__enter__()
  File "/usr/local/miniconda/lib/python3.7/multiprocess

In [2]:
config_path = r'/home/ShareFolder/lgc/Modules/Research/config/feature_bt_template'
print('Loading the configuration from ' + config_path)
configs = namespace.load_namespace(config_path)
FT = FeatureAnalysis(configs, feature_path=r"/home/wuwenjun/factor_lib/")
alpha_list = list(FT.features_in_path.keys())[-1:]
alpha_list = ["10d_ret_test"]

print(alpha_list)
start_date = '2015-01-01'
end_date = '2021-06-01'
FT.load_feature_from_file(alpha_list, start_date, end_date, universe='Investable',
                                       timedelta=None, transformer=norm.standard_scale)
FT.load_return_data()

Loading the configuration from /home/ShareFolder/lgc/Modules/Research/config/feature_bt_template


Loading Feature...:   1%|▊                                                                                          | 14/1559 [00:00<00:39, 39.09it/s]

['10d_ret_test']


Merging feature and return...: 100%|█████████████████████████████████████████████████████████████████████████████| 1559/1559 [00:08<00:00, 179.29it/s]
Process ForkPoolWorker-62:
Process ForkPoolWorker-35:
Process ForkPoolWorker-63:
Process ForkPoolWorker-27:
Process ForkPoolWorker-69:
Process ForkPoolWorker-67:
Process ForkPoolWorker-38:
Process ForkPoolWorker-54:
Process ForkPoolWorker-50:
Process ForkPoolWorker-12:
Process ForkPoolWorker-8:
Process ForkPoolWorker-13:
Process ForkPoolWorker-34:
Process ForkPoolWorker-53:
Process ForkPoolWorker-2:
Process ForkPoolWorker-32:
Process ForkPoolWorker-4:
Process ForkPoolWorker-33:
Process ForkPoolWorker-26:
Process ForkPoolWorker-56:
Process ForkPoolWorker-21:
Process ForkPoolWorker-10:
Process ForkPoolWorker-1:
Process ForkPoolWorker-37:
Process ForkPoolWorker-6:
Process ForkPoolWorker-24:
Process ForkPoolWorker-17:
Process ForkPoolWorker-3:
Process ForkPoolWorker-28:
Process ForkPoolWorker-5:
Process ForkPoolWorker-57:
Process ForkPoolWor

  File "/usr/local/miniconda/lib/python3.7/multiprocessing/process.py", line 297, in _bootstrap
    self.run()
  File "/usr/local/miniconda/lib/python3.7/multiprocessing/process.py", line 297, in _bootstrap
    self.run()
  File "/usr/local/miniconda/lib/python3.7/multiprocessing/process.py", line 297, in _bootstrap
    self.run()
  File "/usr/local/miniconda/lib/python3.7/multiprocessing/process.py", line 297, in _bootstrap
    self.run()
  File "/usr/local/miniconda/lib/python3.7/multiprocessing/process.py", line 297, in _bootstrap
    self.run()
Traceback (most recent call last):
  File "/usr/local/miniconda/lib/python3.7/multiprocessing/process.py", line 297, in _bootstrap
    self.run()
  File "/usr/local/miniconda/lib/python3.7/multiprocessing/process.py", line 297, in _bootstrap
    self.run()
Traceback (most recent call last):
  File "/usr/local/miniconda/lib/python3.7/multiprocessing/process.py", line 297, in _bootstrap
    self.run()
  File "/usr/local/miniconda/lib/python3.7

  File "/usr/local/miniconda/lib/python3.7/multiprocessing/process.py", line 99, in run
    self._target(*self._args, **self._kwargs)
  File "/usr/local/miniconda/lib/python3.7/multiprocessing/process.py", line 297, in _bootstrap
    self.run()
  File "/usr/local/miniconda/lib/python3.7/multiprocessing/process.py", line 99, in run
    self._target(*self._args, **self._kwargs)
  File "/usr/local/miniconda/lib/python3.7/multiprocessing/process.py", line 99, in run
    self._target(*self._args, **self._kwargs)
Process ForkPoolWorker-42:
  File "/usr/local/miniconda/lib/python3.7/multiprocessing/process.py", line 99, in run
    self._target(*self._args, **self._kwargs)
Process ForkPoolWorker-43:
  File "/usr/local/miniconda/lib/python3.7/multiprocessing/process.py", line 99, in run
    self._target(*self._args, **self._kwargs)
  File "/usr/local/miniconda/lib/python3.7/multiprocessing/process.py", line 297, in _bootstrap
    self.run()
  File "/usr/local/miniconda/lib/python3.7/multiproces

  File "/usr/local/miniconda/lib/python3.7/multiprocessing/process.py", line 99, in run
    self._target(*self._args, **self._kwargs)
  File "/usr/local/miniconda/lib/python3.7/multiprocessing/pool.py", line 110, in worker
    task = get()
  File "/usr/local/miniconda/lib/python3.7/multiprocessing/pool.py", line 110, in worker
    task = get()
Traceback (most recent call last):
  File "/usr/local/miniconda/lib/python3.7/multiprocessing/queues.py", line 351, in get
    with self._rlock:
  File "/usr/local/miniconda/lib/python3.7/multiprocessing/queues.py", line 351, in get
    with self._rlock:
  File "/usr/local/miniconda/lib/python3.7/multiprocessing/pool.py", line 110, in worker
    task = get()
  File "/usr/local/miniconda/lib/python3.7/multiprocessing/queues.py", line 351, in get
    with self._rlock:
  File "/usr/local/miniconda/lib/python3.7/multiprocessing/pool.py", line 110, in worker
    task = get()
  File "/usr/local/miniconda/lib/python3.7/multiprocessing/pool.py", line 110

  File "/usr/local/miniconda/lib/python3.7/multiprocessing/synchronize.py", line 95, in __enter__
    return self._semlock.__enter__()
  File "/usr/local/miniconda/lib/python3.7/multiprocessing/queues.py", line 351, in get
    with self._rlock:
  File "/usr/local/miniconda/lib/python3.7/multiprocessing/queues.py", line 351, in get
    with self._rlock:
  File "/usr/local/miniconda/lib/python3.7/multiprocessing/queues.py", line 351, in get
    with self._rlock:
  File "/usr/local/miniconda/lib/python3.7/multiprocessing/process.py", line 297, in _bootstrap
    self.run()
  File "/usr/local/miniconda/lib/python3.7/multiprocessing/queues.py", line 351, in get
    with self._rlock:
  File "/usr/local/miniconda/lib/python3.7/multiprocessing/queues.py", line 351, in get
    with self._rlock:
  File "/usr/local/miniconda/lib/python3.7/multiprocessing/queues.py", line 351, in get
    with self._rlock:
  File "/usr/local/miniconda/lib/python3.7/multiprocessing/synchronize.py", line 95, in __ente

  File "/usr/local/miniconda/lib/python3.7/multiprocessing/queues.py", line 351, in get
    with self._rlock:
  File "/usr/local/miniconda/lib/python3.7/multiprocessing/process.py", line 99, in run
    self._target(*self._args, **self._kwargs)
KeyboardInterrupt
KeyboardInterrupt
KeyboardInterrupt
  File "/usr/local/miniconda/lib/python3.7/multiprocessing/synchronize.py", line 95, in __enter__
    return self._semlock.__enter__()
  File "/usr/local/miniconda/lib/python3.7/multiprocessing/synchronize.py", line 95, in __enter__
    return self._semlock.__enter__()
  File "/usr/local/miniconda/lib/python3.7/multiprocessing/synchronize.py", line 95, in __enter__
    return self._semlock.__enter__()
  File "/usr/local/miniconda/lib/python3.7/multiprocessing/synchronize.py", line 95, in __enter__
    return self._semlock.__enter__()
  File "/usr/local/miniconda/lib/python3.7/multiprocessing/synchronize.py", line 95, in __enter__
    return self._semlock.__enter__()
  File "/usr/local/minicond

In [7]:
final_list = []
for i in tqdm(FT.data_info.keys()):
    one_data = FT.data_info[i].copy()
    one_data.drop("trading_status",axis=1,inplace=True)
    one_data["timestamp"] = i
    one_data.reset_index(inplace=True)
    one_data.set_index(["timestamp","ticker"],inplace=True)
    one_data.columns = [str(i)+"_norm" for i in one_data.columns]
    final_list.append(one_data)
final_list = pd.concat(final_list,axis=0)
final_list.drop("10d_ret_test_norm",axis=1,inplace=True)
final_list

100%|██████████| 1559/1559 [00:12<00:00, 122.85it/s]


5m_ret_norm  15m_ret_norm  30m_ret_norm  \
timestamp           ticker                                            
2015-02-25 09:30:00 600652    -0.003534      0.034158      0.089517   
                    002208    -0.006928     -0.006928      0.010393   
                    600711     0.009333      0.022667      0.024000   
                    300216     0.030000      0.035600      0.039400   
                    000976     0.000000      0.000000      0.000000   
...                                 ...           ...           ...   
2021-05-31 09:30:00 603990    -0.008053      0.000288     -0.009779   
                    603317    -0.012531     -0.018251     -0.024516   
                    000056    -0.009524     -0.026190     -0.033333   
                    300677    -0.005555     -0.000278      0.010276   
                    002612    -0.011374      0.028743      0.023363   

                            60m_ret_norm  120m_ret_norm  1d_ret_norm  \
timestamp           ticker                                             
2015-02-25 09:30:00 600652      0.095406       0.094229     0.081272   
                    002208      0.004619       0.004619     0.004619   
                    600711      0.021333       0.032000     0.114667   
                    300216      0.089800       0.097600     0.100000   
                    000976      0.000000       0.000000     0.099206   
...                                  ...            ...          ...   
2021-05-31 09:30:00 603990      0.009491       0.021283     0.064136   
                    603317     -0.011441      -0.022337    -0.011986   
                    000056     -0.038095       0.002381    -0.014286   
                    300677      0.007916      -0.003194    -0.013887   
                    002612      0.042730       0.057024     0.080695   

                            2d_ret_norm  3d_ret_norm  4d_ret_norm  \
timestamp           ticker                                          
2015-02-25 09:30:00 600652     0.104829     0.247350     0.341578   
                    002208     0.011547     0.177829     0.218245   
                    600711     0.201333     0.360000     0.406667   
                    300216     0.077600     0.097600     0.196000   
                    000976     0.208333     0.329365     0.462302   
...                                 ...          ...          ...   
2021-05-31 09:30:00 603990    -0.010641    -0.085418    -0.079666   
                    603317    -0.000272    -0.037864    -0.060202   
                    000056    -0.090476    -0.190476    -0.154762   
                    300677    -0.179836    -0.243230    -0.257048   
                    002612     0.022133    -0.000922    -0.000922   

                            5d_ret_norm  10d_ret_norm  20d_ret_norm  
timestamp           ticker                                           
2015-02-25 09:30:00 600652     0.309776      0.936396      1.204947  
                    002208     0.218245      0.785219      0.891455  
                    600711     0.640000      0.817333      1.040000  
                    300216     0.233000      0.780000      0.574000  
                    000976     0.531746      0.650794      0.603175  
...                                 ...           ...           ...  
2021-05-31 09:30:00 603990    -0.137187     -0.226920     -0.267472  
                    603317    -0.081994     -0.229365     -0.223645  
                    000056    -0.230952     -0.283333     -0.350000  
                    300677    -0.259131     -0.300097     -0.197334  
                    002612    -0.119428     -0.277590     -0.146941  

[4765522 rows x 12 columns]

In [8]:
from AlphaNet.Data import convert_to_standard_daily_data_par
for i in tqdm(final_list.columns):
    convert_to_standard_daily_data_par(df=pd.DataFrame(final_list[i]),output_name = i,output_path="/home/wuwenjun/feature_platform/ti0/wuwenjun")

  0%|          | 0/12 [00:00<?, ?it/s]

Loading the configuration from /home/ShareFolder/lgc/Modules/Research/config/feature_bt_template
Loading the configuration from /home/ShareFolder/lgc/Modules/Research/config/feature_bt_template


100%|██████████| 12/12 [00:45<00:00,  3.83s/it]
